In [1]:
from shutil import which
print(which('espeak'))
print(which('espeak-ng'))

/usr/bin/espeak
None


In [9]:
!nvidia-smi
import os
# import gdown
import pandas as pd
import soundfile as sf
import shutil
import bangla
import torch
import pysbd
import numpy as np

try:
    from TTS.utils.synthesizer import Synthesizer
except:
    print("coundn't import TTS synthesizer,trying again!")

import TTS
from typing import List

from TTS.config import load_config
from TTS.tts.models import setup_model as setup_tts_model

from TTS.tts.utils.synthesis import synthesis, transfer_voice, trim_silence
from TTS.utils.audio import AudioProcessor
from TTS.vocoder.models import setup_model as setup_vocoder_model
from TTS.vocoder.utils.generic_utils import interpolate_vocoder_input
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.models.vits import Vits, VitsAudioConfig
from TTS.tts.utils.text.tokenizer import TTSTokenizer

from trainer import Trainer, TrainerArgs

from TTS.tts.configs.shared_configs import BaseDatasetConfig,BaseAudioConfig,CharactersConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.utils.speakers import SpeakerManager
import time

Thu Jan 11 17:59:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    On  | 00000000:00:1E.0 Off |                    0 |
|  0%   24C    P0              57W / 300W |    582MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
male = True
pretrained = True # False
run_name = "VITS"
batch_size = 48
eval_batch_size = 48
num_workers = 16 # number of CPU cores
epochs = 10000 # number of epoch
save_step = 10000
print_step = 1000

In [10]:
pretrained_path = ''
if(pretrained):
    pretrained_path = 'VITS-February-06-2024_11+33PM-0000000'
if(male):
    meta_file = 'content/iitm_bangla_tts/comprehensive_bangla_tts/male/mono/metadata_male.txt'
    root_path = 'content/iitm_bangla_tts/comprehensive_bangla_tts/male/mono'
else:
    meta_file = 'content/iitm_bangla_tts/comprehensive_bangla_tts/female/mono/metadata_female.txt'
    root_path = 'content/iitm_bangla_tts/comprehensive_bangla_tts/female/mono'

def formatter(root_path, meta_file, **kwargs):  # pylint: disable=unused-argument
    """Normalizes the LJSpeech meta data file to TTS format
    https://keithito.com/LJ-Speech-Dataset/"""
    txt_file = meta_file
    items = []
    speaker_name = "ljspeech"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = os.path.join(root_path, "wav", cols[0] + ".wav")
            try:
                text = cols[1]
            except:
                print("not found")

            items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_name, "root_path": root_path})
    return items


dataset_config = BaseDatasetConfig(
     meta_file_train=meta_file, path=os.path.join(root_path, "")
)

In [11]:
train_samples, eval_samples = load_tts_samples(dataset_config,formatter=formatter, eval_split=True)

 | > Found 6187 files in /home/ec2-user/SageMaker/content/iitm_bangla_tts/comprehensive_bangla_tts/male/mono


In [12]:
output_path = ''

audio_config = VitsAudioConfig(
    sample_rate=22050, 
    win_length=1024, 
    hop_length=256, 
    num_mels=80, 
    mel_fmin=0, 
    mel_fmax=None
)


if(male):
    characters_config = CharactersConfig(
    pad = '<PAD>',
    eos = '।', #'<EOS>', #'।',
    bos = '<BOS>',# None,
    blank = '<BLNK>',
    phonemes = None,
    characters =  "তট৫ভিঐঋখঊড়ইজমএেঘঙসীঢ়হঞ‘ঈকণ৬ঁৗশঢঠ\u200c১্২৮দৃঔগও—ছউংবৈঝাযফ\u200dচরষঅৌৎথড়৪ধ০ুূ৩আঃপয়’নলো",
    punctuations = "-!,|.? ",
    )
else:
    characters_config = CharactersConfig(
    pad = '<PAD>',
    eos = '।', #'<EOS>', #'।',
    bos = '<BOS>',# None,
    blank = '<BLNK>',
    phonemes = None,
    characters =  "ইগং়’ুঃন১ঝূও‘ঊোছপফৈ৮ষযৎঢঈকঠিজ০৬ীটডএঅঋধচে২৩ণউয়ঢ়খলভৗসহ্ড়দথবঔাঞশরৌম—ঐআৃঘঙ\u200cঁ৪৫ত",
    punctuations = ".?-!|, ",
    )

In [13]:
# VitsConfig: all model related values for training, validating and testing.

config = VitsConfig(
    audio = audio_config,
    run_name = run_name,
    batch_size = batch_size,
    eval_batch_size = eval_batch_size,
    batch_group_size = 0,
    num_loader_workers = num_workers,
    num_eval_loader_workers = num_workers,
    run_eval = True,
    test_delay_epochs = -1,
    epochs = epochs,
    text_cleaner = None,
    use_phonemes = True,
    phoneme_language = "bn",
    phoneme_cache_path = os.path.join(output_path, "bn_phoneme"),
    compute_input_seq_cache = True,
    print_step = print_step,
    print_eval = False,
    mixed_precision = True,
    output_path = output_path,
    datasets = [dataset_config],
    characters = characters_config,
    save_step = save_step,
    cudnn_benchmark = True,
    test_sentences = [
        'হয়,হয়ে,ওয়া,হয়েছ,হয়েছে,দিয়ে,যায়,দায়,নিশ্চয়,আয়,ভয়,নয়,আয়াত,নিয়ে,হয়েছে,দিয়েছ,রয়ে,রয়েছ,রয়েছে।',
        'দেয়,দেওয়া,বিষয়,হয়,হওয়া,সম্প্রদায়,সময়,হয়েছি,দিয়েছি,হয়,হয়েছিল,বিষয়ে,নয়,কিয়াম,ইয়া,দেয়া,দিয়েছে,আয়াতে,দয়া।',
        'ইয়াহুদ,নয়,ব্যয়,ইয়াহুদী,নেওয়া,উভয়ে,যায়,হয়েছিল,প্রয়োজন।'
    ]
)

In [14]:
ap = AudioProcessor.init_from_config(config)

tokenizer, config = TTSTokenizer.init_from_config(config)

model = Vits(config, ap, tokenizer, speaker_manager=None)

trainer = Trainer(
    TrainerArgs(continue_path = pretrained_path),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples
)

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


fatal: not a git repository (or any parent up to mount point /home/ec2-user)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent up to mount point /home/ec2-user)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 16
 | > Num. of Torch Threads: 8
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: True
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=vits_11_jan-January-11-2024_05+59PM-0000000

 > Model has 83050732 parameters


In [ ]:
trainer.config.test_sentences = [
        'হয়,হয়ে,ওয়া,হয়েছ,হয়েছে,দিয়ে,যায়,দায়,নিশ্চয়,আয়,ভয়,নয়,আয়াত,নিয়ে,হয়েছে,দিয়েছ,রয়ে,রয়েছ,রয়েছে।',
        'দেয়,দেওয়া,বিষয়,হয়,হওয়া,সম্প্রদায়,সময়,হয়েছি,দিয়েছি,হয়,হয়েছিল,বিষয়ে,নয়,কিয়াম,ইয়া,দেয়া,দিয়েছে,আয়াতে,দয়া।',
        'ইয়াহুদ,নয়,ব্যয়,ইয়াহুদী,নেওয়া,উভয়ে,যায়,হয়েছিল,প্রয়োজন।'
    ]

In [ ]:
# let's 🚀

trainer.fit()


 > EPOCH: 0/100
 --> vits_11_jan-January-11-2024_05+59PM-0000000




> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: bn
		| > phoneme backend: bn_phonemizer
| > Number of instances : 6126



 > TRAINING (2024-01-11 17:59:55) 


 | > Preprocessing samples
 | > Max text length: 114
 | > Min text length: 16
 | > Avg text length: 64.72233104799217
 | 
 | > Max audio length: 276735
 | > Min audio length: 49452
 | > Avg audio length: 129085.73326803787
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.
